# Define the dataset

In [1]:
import pandas as pd
df = pd.DataFrame({
    'StorageTemperature': [2, 8, 15, 25],
    'TotalMushrooms': [30,25,20,30],
    'SpoiledMushrooms': [2,4,5,20]
})
df

,StorageTemperature,TotalMushrooms,SpoiledMushrooms
0,2,30,2
1,8,25,4
2,15,20,5
3,25,30,20


# Modelling assumptions:
1. The outcome of the \($n_i$\) mushrooms within each group \(i\) are *independent*.  
   Each mushroom in the group has probability \($p_i$\) of death.

2. The probability \($p_i$\) that a mushroom spoils depends on the temperature level \($x_i$\) as follows:

   $$
   p_i = \text{sigm}(\alpha + \beta x_i)
   $$

   where

   $$
   \text{sigm}(z) = \frac{1}{1 + e^{-z}}
   $$

3.  The parameters $\theta = [\alpha, \beta]^\top$ have independent Gaussian priors:
    \begin{align}
    \alpha &\sim \mathcal{N}(\mu_\alpha, \sigma_\alpha^2), \quad \mu_\alpha = 0, \sigma_\alpha = 2 \\
    \beta &\sim \mathcal{N}(\mu_\beta, \sigma_\beta^2), \quad \mu_\beta = 0, \sigma_\beta = 1
    \end{align}

4.  The outcomes in the four groups are independent of each other, given $𝜃$.


## 1.1: Probabilistic model

* Derive and comment the full probabilistic model.

## Bayesian Logistic Regression Model

We aim to model the spoilage of mushrooms based on temperature.

### 1. Variables
* $i$: Index for the experimental group.
* $n_i$: Total number of mushrooms in group $i$.
* $y_i$: Number of spoiled mushrooms in group $i$.
* $x_i$: Temperature level for group $i$.

### 2. Likelihood
The outcome of the $n_i$ mushrooms within each group $i$ is independent. The number of spoiled mushrooms follows a Binomial distribution:

$$y_i \sim \text{Binomial}(n_i, p_i)$$
where $p_i$ is the probability of spoilage in group $i$ and $n_i$ is the total number of mushrooms in group $i$.


The probability $p_i$ that a mushroom spoils depends on the temperature $x_i$ so we can model the log odds of spoilage as a linear function of temperature:

$$sigmoid(z) = \frac{1}{1 + e^{-z}}$$
so in our case:

$$p_i = \text{sigmoid}(\alpha + \beta x_i) = \frac{1}{1 + e^{-(\alpha + \beta x_i)}}$$


### 3. Prior
The parameters $\theta = [\alpha, \beta]^\top$ have independent Gaussian priors:

$$ \alpha \sim \mathcal{N}(0, 2^2) $$
$$ \beta \sim \mathcal{N}(0, 1^2) $$

**Derivation of the joint prior density:**
Due to independece the joint prior density is the product of the individual priors:
$$ f(\theta) = f(\alpha, \beta) = f(\alpha) \cdot f(\beta)$$

Moreover, given $ (\alpha, \beta) $ the outcomes in the four groups are independent of each other.

$$
P(\mathbf{y} \mid \alpha, \beta) = \prod_{i=1}^4 P(y_i \mid n_i, x_i, \alpha, \beta)
$$

Substituting the standard normal PDF formula $$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$ with our specific hyperparameters ($\sigma_\alpha=2, \sigma_\beta=1, \mu=0$):

$$ \begin{aligned}
f(\alpha, \beta) &= \left( \frac{1}{\sqrt{2\pi \cdot 4}} e^{-\frac{\alpha^2}{2 \cdot 4}} \right) \cdot \left( \frac{1}{\sqrt{2\pi \cdot 1}} e^{-\frac{\beta^2}{2 \cdot 1}} \right) \\
&= \left( \frac{1}{2\sqrt{2\pi}} \cdot \frac{1}{\sqrt{2\pi}} \right) \cdot \exp\left( -\frac{\alpha^2}{8} - \frac{\beta^2}{2} \right) \\
&= \frac{1}{4\pi} \exp\left( \frac{-\alpha^2}{8} - \frac{4\beta^2}{8} \right)
\end{aligned}
$$



**Final Joint Prior:**
$$
f(\alpha, \beta) = \frac{1}{4\pi} \exp \left( -\frac{\alpha^2 + 4\beta^2}{8} \right)
$$


## 1.2: Maximum Likelihood estimation
* Derive an analytical expression of the likelihood function $\mathcal{L}(\theta) = P(y|\theta)$.

The total likelihood function is just the product of the likelihoods of each group:

$$
\mathcal{L}(\vec{\theta}) = \prod_{i=1}^{n} \binom{n_i}{y_i} \cdot p_i^{y_i} \cdot (1-p_i)^{n_i-y_i}
$$
For each group $i$, the likelihood of observing $y_i$ spoiled mushrooms out of $n_i$ given the spoilage probability $p_i$ is given by the Binomial distribution formula

Substituting $p_i = \mathrm{sigm}(\alpha + \beta x_i)$:

$$
= \prod_{i=1}^{n} \binom{n_i}{y_i} \cdot \mathrm{sigm}(\alpha + \beta x_i)^{y_i} \cdot [1-\mathrm{sigm}(\alpha + \beta x_i)]^{n_i-y_i}
$$


$$
\propto \prod_{i=1}^{n} \mathrm{sigm}(\alpha + \beta x_i)^{y_i} \cdot [1-\mathrm{sigm}(\alpha + \beta x_i)]^{n_i-y_i}
$$

We can ignore the binomial coefficients $\binom{n_i}{y_i}$ since they do not depend on the parameters $\theta = [\alpha, \beta]^\top$ and thus do not affect the location of the maximum likelihood estimate.
They do not depend because they are constants with respect to $\alpha$ and $\beta$.

n is the number of groups (4 in this case) NOT the number of samples.

* Derive an analytical expression of the log-likelihood function $\ell(\theta)$

Multiplying several probability mass function leads to very small numbers that can cause numerical underflow, using the log-likelyhood we ensure numerical stability, in fact a log of a product becames a sum which ensures we don't go towards zero.

Moreover, it is often easier to maximize an analytical expression using the logarithm since it is monotonically increasing, maximizing $f(x)$ is equivalent to maximize $f(g(x))$ in fact the location of the maximum doesn't change

$$
\begin{aligned}
\mathcal{\ell}(\vec{\theta}) &= \log \mathcal{L}(\vec{\theta}) \\
&= \sum_{i=1}^{m} \Bigg( \log \binom{n}{y} + \log \left( \mathrm{sigm}(\alpha x + \beta)^y \right) + \log \left( (1-\mathrm{sigm}(\alpha x + \beta))^{n-y} \right) \Bigg) \\
&= \sum_{i=1}^{m} \Bigg( \log \binom{n}{y} + y \log \left( \mathrm{sigm}(\alpha x + \beta) \right) + (n-y) \log \left( 1-\mathrm{sigm}(\alpha x + \beta) \right) \Bigg) \\
&\propto \sum_{i=1}^{m} \Bigg( y \log \left( \mathrm{sigm}(\alpha x + \beta) \right) + (n-y) \log \left( 1-\mathrm{sigm}(\alpha x + \beta) \right) \Bigg)
\end{aligned}
$$


Write a Python function corresponding to the log-likelihood function ℓ(𝜃) (possibly up
to an additive factor)

In [ ]:
import numpy as np
def sigmoid(z):
    return 1/(1 - np.exp(-z))
